# Advent of Code - 2024 - Day 6 - Problem 2

https://adventofcode.com/2024/day/5

## Load Source Data

Load the map data into `DATA`.

In [54]:
f = open("data/day6.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

## Create Maze Class

In [61]:
class Maze:

    def __init__(self, data):
        self.data = list(map(list, data))

    def is_valid(self, location):
        row, col = location
        if row < 0 or row >= len(self.data): return False
        if col < 0 or col >= len(self.data[row]): return False
        return True

    def get_value(self, location):
        row, col = location
        return self.data[row][col]
    
    def find_value(self, value):
        for row in range(len(self.data)):
            for col in range(len(self.data[row])):
                if self.data[row][col] == value: return (row,col)
        return None

## Create Cursor Class

In [56]:
ORIENTATION_UP = 0
ORIENTATION_RIGHT = 1
ORIENTATION_DOWN = 2
ORIENTATION_LEFT = 3
class Cursor:

    def __init__(self, location, orientation):
        self.location = location
        self.orientation = orientation

    def __str__(self):
        return f"({self.location[0]},{self.location[1]},{self.orientation})"

    # STATIC
    def get_next_location(self, location, orientation):

        row, col = location

        if   orientation == ORIENTATION_UP:    return (row-1, col)
        elif orientation == ORIENTATION_RIGHT: return (row  , col+1)
        elif orientation == ORIENTATION_DOWN:  return (row+1, col)
        else:                                  return (row  , col-1) # LEFT

    def get_next_locations(self):
        location = self.location
        orientation = self.orientation
        for idx in range(4):
            yield Cursor(self.get_next_location(location, orientation), orientation)
            orientation = (orientation + 1) % 4

## Create Guard Class

In [ ]:
import copy

GUARD_UP = "^"
GUARD_RIGHT = ">"
GUARD_DOWN = "v"
GUARD_LEFT = "<"
ALL_GUARDS = [GUARD_UP, GUARD_RIGHT, GUARD_DOWN, GUARD_LEFT]

GUARD_VISITED = "X"

class MoveFailure(Exception):
    pass

class Guard:

    def __init__(self, maze, cursor):
        self.maze = maze
        self.cursor = cursor
        self.path = list()
        self.str_path = set()
        
        self.path.append(cursor)
        self.str_path.add(str(cursor))

    def clone(self):
        g = Guard(self.maze, self.cursor)
        g.path = self.path.copy()
        g.str_path = self.str_path.copy()
        return g

    def move(self, escaping):

        for next_cursor in self.cursor.get_next_locations():
            
            # See if we've moved off the map.
            #
            if not self.maze.is_valid(next_cursor.location): 
                return True

            # Move if the direction isn't blocked.
            #
            value = self.maze.get_value(next_cursor.location)
            if value != "#":

                if str(next_cursor) in self.str_path:
                    raise MoveFailure("Loop detected.")
                
                if not escaping:
                    self.can_escape()

                self.cursor = next_cursor
                self.path.append(self.cursor)
                self.str_path.add(str(self.cursor))

                return False
            
        raise MoveFailure("No possible moves.")
    
    def escape(self):
        while (not self.move(True)):
            pass

    def can_escape(self):
        g = copy.copy(self)
        g.escape()
        return True
    
    def search(self):
        while (not self.move(False)):
            pass



## Move Guard Around Maze

In [ ]:
maze = Maze(DATA)

location = maze.find_value(GUARD_UP)
cursor = Cursor(location, ORIENTATION_UP)

guard = Guard(maze, cursor)
guard.search();

s = set()
s.update([f"{c.location[0]},{c.location[1]}" for c in guard.path])
print(len(s))

# print(len(guard.path))

# for line in maze.data:
#    for char in line:
#       if char in [GUARD_VISITED]: count += 1

# print(guard.path)
# c = Cursor((37,46),0)
# print(c)
# print([str(x) for x in guard.path if str(x) == str(c)])#x.location == c.location and x.orientation == c.orientation])